## Add provider groups to our SQL database

Lets first import some of the basic packages we will be using.

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl
import glob
from ast import literal_eval

# Files we need to obtain

We need to first obtain the files that have our provider groups. I have stored the parent directory in a dotenv locally, but feel free to create your own variables to replace the ones here.

In [2]:
load_dotenv()

hyperlink_path = 'json_completed_hyperlinks_update.csv'
parent_dir = os.getenv('dir')
data_dir = os.path.join(parent_dir,'data_update')

df = pd.read_csv(hyperlink_path, header=None)
df.head()
df.columns = ['ParseID','Hyperlink']
hyperlinks = df['Hyperlink'].tolist()

def foldername(hyperlink):
    hyperlink = hyperlink.split('/')[-1]
    return hyperlink[0:-8]
def providers_path(folder):
    return os.path.join(data_dir,folder,folder+'_providers.csv')

folder_names= [foldername(hyperlink) for hyperlink in hyperlinks]
provider_files = [providers_path(folder_name) for folder_name in folder_names]

provider_files[0]

'D:/Vignesh/Capstone\\data_update\\2023-01-01_ALL-SAVERS-INSURANCE-COMPANY_Insurer_PPO---NDC_PPO-NDC_in-network-rates\\2023-01-01_ALL-SAVERS-INSURANCE-COMPANY_Insurer_PPO---NDC_PPO-NDC_in-network-rates_providers.csv'

# Lets read one:

Let us read one file to see what we are dealing with.

In [3]:
df = pd.read_csv(provider_files[0], usecols=['tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval})
folder = folder_names[0]

In [4]:
df.dtypes
df.head()

,tin,npi_provider_groups
0,593582520,[1225090087]
1,272050459,[1639508567]
2,160743209,[1609314343]
3,561844651,"[1215134309, 1043649635, 1851721047, 1285715185]"
4,371756970,[1174710636]


Notice how we have an array of npi numbers, lets explode this.

In [5]:
df_explode = df.explode(column= 'npi_provider_groups',ignore_index=True)
df_explode.head()

,tin,npi_provider_groups
0,593582520,1225090087
1,272050459,1639508567
2,160743209,1609314343
3,561844651,1215134309
4,561844651,1043649635


# Lets add all the files to a single dataframe

In [26]:
df = pd.concat((pd.read_csv(f, usecols=['tin','npi_provider_groups'], converters={'npi_provider_groups': literal_eval}) for f in provider_files), ignore_index=True)




In [27]:
df.describe

<bound method NDFrame.describe of                   tin                               npi_provider_groups
0         593582520.0                                      [1225090087]
1         272050459.0                                      [1639508567]
2         160743209.0                                      [1609314343]
3         561844651.0  [1215134309, 1043649635, 1851721047, 1285715185]
4         371756970.0                                      [1174710636]
...               ...                                               ...
11041685  460815208.0              [1205114162, 1447820287, 1629482070]
11041686  208246007.0                                      [1386861011]
11041687  461649191.0              [1538565510, 1376294009, 1114211968]
11041688  832105550.0                                      [1124401732]
11041689  271562642.0                          [1487756649, 1497881734]

[11041690 rows x 2 columns]>

In [28]:
df.drop_duplicates(subset=['tin'], inplace=True, ignore_index=True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46855 entries, 0 to 46854
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   tin                  46854 non-null  float64
 1   npi_provider_groups  46855 non-null  object 
dtypes: float64(1), object(1)
memory usage: 732.2+ KB


In [ ]:
# df = df.explode(column= 'npi_provider_groups',ignore_index=True)

In [35]:
npi_path = 'Facility_Affiliation.csv'
npi = pd.read_csv(npi_path,usecols=['NPI','facility_afl_ccn','parent_ccn'], encoding='windows-1252')

In [36]:
npi.head()

,NPI,facility_afl_ccn,parent_ccn
0,1003000126,210003,NaN
1,1003000126,210022,NaN
2,1003000126,490063,NaN
3,1003000126,490145,NaN
4,1003000134,140010,NaN


In [39]:
npi['parent_ccn']= npi.apply(lambda row: row.facility_afl_ccn if row.parent_ccn is np.nan else row.parent_ccn, axis=1)

KeyboardInterrupt: 